In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

from herbie.scale import Scale
from herbie.generator import Generator
import herbie.filters as filters
from nb_utils import play, to_wav

# Dani California

## Bass

In [ ]:
bass = Generator()
bass.fn = lambda t: np.sin(t) + 0.1 * np.sin(2*t)
attack = (1, 1/20)
decay = (0.3, 1/8)
release = (0, 1/4)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.96, 0.5)
bass.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=1, bpm=90)
am.default_note = 8
b1 = am.sequence("A A A^ r.16 Ev.16")
b2 = am.sequence("Gv Gv G F")
b3 = am.sequence("D D D^ r.16 D.16")
b4 = am.sequence("G.16 A^.16 G E D")
score = b1 + b2 + b3 + b4
bassline = bass.sequence(score)
bassline = np.tile(bassline, 3)
play(bassline, autoplay=False)

## Octave bass

In [ ]:
bass2 = Generator()
attack = (1, 1/20)
decay = (0.3, 1/8)
release = (0, 1/4)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.5, 0.5)
bass2.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=2, bpm=90)
am.default_note = 8
b1 = am.sequence("A A A^ r.16 Ev.16")
b2 = am.sequence("Gv Gv G F")
b3 = am.sequence("D D D^ r.16 D.16")
b4 = am.sequence("G.16 A^.16 G E D")
score = b1 + b2 + b3 + b4
bassline2 = bass2.sequence(score)
bassline2 = np.tile(bassline2, 3)
play(bassline2, autoplay=False)

## Chords

In [ ]:
rhy = Generator()
attack = (1, 1/16)
decay = (0.8, 1/8)
release = (0, 1/8)
adsr = lambda s: filters.adsr(s, attack, decay, release)
clip = lambda s: filters.clip(s, 0.8, 0.2)
rhy.filters = [clip, adsr]

In [ ]:
am = Scale("A", "minor", octave=4, bpm=90)
b1 = am.sequence("A-C-E.2")
b2 = am.sequence("Gv-B-D.2")
b3 = am.sequence("D-F-A.2")
b4 = am.sequence("F-A-C.2")
score = b1 + b2 + b3 + b4
chords = rhy.sequence(score)
chords = np.tile(chords, 3)
play(chords, autoplay=False)

## Mash em all together

In [ ]:
from typing import List
def masher(parts: List[np.array]) -> np.array:
    mlen = np.max([len(p) for p in parts])
    padded = []
    for p in parts:
        padded.append(np.pad(p, (0, mlen - len(p))))
    mashed = np.sum(padded, axis=0)
    return filters.normalize(mashed)

In [ ]:
song = np.array([masher([bassline*0.5, bassline2, chords*0.3]), masher([chords*0.3, bassline])])
play(song)

In [ ]:
to_wav("out/dani_california.wav", song.T)